In [ ]:
# Initialize Otter
import otter
grader = otter.Notebook("lab07.ipynb")

# Lab 7: Exploiting Weak Randomness
Contributions From: Ryan Cottone

Welcome to Lab 7! In this lab, we will examine different PRNG setups, learn how to measure the randomness of something, and take a look at how simple breaking something can be.

In [ ]:
%%capture
import sys
!{sys.executable} -m pip install pycryptodome

In [ ]:
from Crypto.Cipher import AES
from Crypto.Util.Padding import pad
from Crypto.Hash import SHA256

In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

In [ ]:
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)

def modularInverse(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception('modular inverse does not exist')
    else:
        return x % m

def returnCounts(generator, possibleOutputs):
    d = dict()
    
    for output in possibleOutputs:
        d[output] = 0
    
    for output in iter(generator):
        d[output] +=1
        
    return d

# Random Number Generation

The need for random numbers is omnipresent within cryptography -- mainly key generation, but also other uses such as choosing primes for public key cryptosystems and salts for hashing. Without a way to randomly pick something, attackers could always predict what the keys would be!

However, by their deterministic nature, computers are utterly incapable of being random. There exist algorithms (pseudorandom number generators) to generate kind-of random numbers (pseudorandom), but all rely on some early, random input to derive the later outputs. Therefore, if an attacker can compromise the internal state of a PRNG, they can predict all of its future outputs by just running their own version of the PRNG with that state.

## Cryptographically-Secure Random Number Generators

CSPRNGs are a special class of PRNGs that are (relatively) safe to use for cryptography. These PRNGs satisfy the following criteria:

- Output is computationally indistinguishable from true randomness
- Difficult to leak the state
- Rollback resistant (hard to find outputs before time **t** given the state at time **t**

**Never use a normal PRNG for cryptography!**

## Linear Congruential Generator

A simple example of a PRNG (NOT cryptographically secure) is the **linear congruential generator**, which has the following function to output a new number, given the previous number $s$:

$$ f(s) = \alpha s + \beta \mod n $$

For example, if we had $\alpha = 5, \beta = 2, n = 7$, then we would start at 1, then output $5(1) + 2 \mod 7 = 0$, etc.

Were somebody to **incorrectly** use this as a CSPRNG, they would try to keep $\alpha$ and $\beta$ secret, in order to try and prevent an attacker from learning the state by observing outputs. We will see that even this is not effective.

**Question 1**: Implement the linear congruential generator!

In [ ]:
def LCG(alpha, beta, n, count):
    assert math.gcd(alpha, n) == 1, "Alpha must be relatively prime to n"
    
    output = 1 # This is the initial output of the program
    
    while count > 0:
        # Set next output equal to alpha * output + beta mod n
        output = ...
        yield output # This lets us use python generators for easier functionality. Don't worry about this line!
        count -= 1

In [ ]:
grader.check("q1_1")

Let's plot out what the output of our LCG looks like over many iterations:

In [ ]:
plt.bar([0,1,2,3,4,5,6,7], returnCounts(LCG(3, 3, 8, 1000), range(8)).values())
plt.show()

That doesn't look very random! The reason we are only outputting 1,2,5, and 6 is due to the theoretical **period** of the linear congruential generator. The period of an RNG is the length of output it can achieve before starting at the beginning (equivalently, how many different outputs it can have). While out of scope, a LCG will have period **n** (i.e., uniformly output every number $[0, n-1]$ if and only if:

$$\begin{align*}
\gcd(\beta, n) = 1\\
\text{Each prime divisor of p divides } a-1\\
(4 \mid n) \implies (4 \mid a-1)
\end{align*}$$

The proof of which can be found [here](https://www.staff.uni-mainz.de/pommeren/Cryptology/Bitstream/1_Classic/Maxper.pdf) if you are interested.

In [ ]:
plt.bar([0,1,2,3,4,5,6,7], returnCounts(LCG(5, 3, 8, 1000), range(8)).values())
plt.show()

We see that a fixed version of $\alpha = 5, \beta = 3, n = 8$ fits these requirements and therefore outputs a uniform distribution.

## Breaking the Linear Congruential Generator

This is a nice and simple PRNG, but fundamentally flawed for cryptographic work. Given even a few outputs, it is possible to recover $n$. Once we have $n$, it is straightforward to figure out $\alpha$ and $\beta$.

Say we knew $n$ and had outputs $s_1$, $s_2$, and $s_3$. Then,

$$s_2 \equiv \alpha s_1 + \beta \mod n$$
$$s_3 \equiv \alpha s_2 + \beta \mod n$$

which is 2 linear equations and 2 unknowns. This can be solved with normal algebra to find our unknowns, and therefore break the entire system!

So, we know that finding $n$ immediately breaks the system. But how do we find $n$?

It turns out that there are quite efficient methods that do this with as little as 9 outputs from the LCG. That method uses determinants and is out of scope, but there are also methods relying on finding multiples of $n$ and using gcd (also out of scope). For now, we can just find the maximum number output by the LCG, and add one to find $n$. Of course, this assumes a uniform output LCG, but if that were not true, our job is even easier.

In [ ]:
plt.bar([0,1,2,3,4,5,6,7,8,9,10], list(returnCounts(LCG(5, 3, 8, 1000), range(8)).values()) + [0,0,0])
plt.show()

Take our earlier LCG for example. Visually, we can easily see that the numbers max out at 7. That implies the modulus must be 8.

Solving the following system of equations:

$$3 \equiv \alpha (0) + \beta \mod 8$$
$$2 \equiv \alpha (2) + \beta \mod 8$$

we figure out that $\alpha = 5, \beta = 3$ as expected!

## Rollback Resistance

Next, we will see why the LCG has no rollback resistance once $\alpha$ and $\beta$ are discovered. By rewriting our initial equation, we see that 

$$
\begin{align*}
s_{i+1} \equiv \alpha s_i + \beta \mod n \\
(s_{i+1} - \beta) \equiv \alpha s_i \mod n \\
\alpha^{-1}(s_{i+1} - \beta) \equiv s_i \mod n
\end{align*}
$$

thereby completing revealing $s_i$ given $s_{i+1}$.

**Question 1.2**: Implement a function that, given $\alpha$, $\beta$, $n$, and one output $s_i$ recovers the previous  output.

In [ ]:
def recoverPreviousOutput(alpha, beta, n, s):
    inverseOfAlpha = modularInverse(alpha, n)
    
    # Remember the formula to find the previous value of s!
    ...

In [ ]:
grader.check("q1_2")

Now, let's take a look at a real-world attack against someone using the LCG! Say Alice and Bob are trying to communicate and need to establish a secret key. They use a LCG to generate their secret keys $a, b$ and send $g^a \mod N$, $g^b \mod N$ respectively. We, the attacker, are eventually able to recover three outputs after $a,b$, named $s_3$, $s_4$, and $s_5$.

Using the properties we explored above, we are able to completely compromise their communications and learn the secret $g^{ab} \mod N$.

In [ ]:
# Solves a 2-equation system of alpha * val + beta = val[1] mod N ...
# Vals must be three different outputs of the system
# Brute force because its only two variables and not a very large modulus
def solveModularSystem(vals, N):
    for alpha in range(N):
        for beta in range(N):
            if (alpha*vals[0] + beta)%N == vals[1] and (alpha*vals[1] + beta)%N == vals[2]:
                return alpha,beta
    return None, None

In [ ]:
# Don't worry about this
def encrypt(key,plaintext):
    key = bytes(key)
    plaintext = plaintext[:21]
    cipher = AES.new(pad(key, 16), AES.MODE_ECB) # Don't use ECB in real applications! This is just for ease of coding
    
    return cipher.encrypt(pad(plaintext, AES.block_size))

# Don't worry about this
def decrypt(key,ciphertext):
    key = bytes(key)
    cipher = AES.new(pad(key, 16), AES.MODE_ECB)
    
    try:
        return cipher.decrypt(pad(ciphertext, AES.block_size))[:21].decode('utf-8')
    except UnicodeDecodeError:
        return 'ERROR DECODING MESSAGE: LIKELY INCORRECT KEY'

# Don't worry about this
def H(x):
    if type(x) != bytes:
        if type(x) == str:
            x = bytes(x, 'utf-8')
        elif type(x) == int:
            x = x.to_bytes(len(bin(x))-2, byteorder='big')
    
    h = SHA256.new()
    h.update(x)
    return h.digest().hex()

**Question 2.1**: Break this faulty Diffie-Hellman system!

In [ ]:
N = 53 # The modulus used in Diffie Hellman and the LCG
g = 2 # The generator used in Diffie Hellman

g_a = 14 # g^a mod N
g_b = 48 # g^b mod N

s_3 = 16 # The third output of the LCG
s_4 = 29 # The fourth output of the LCG
s_5 = 27 # The fifth output of the LCG

# You can assume the modulo used in the LCG is also 53.
# Use solveModularSystem() to find alpha and beta!
# Pass in an array of three outputs and the modulus and receive (alpha, beta)

# Start with recoveredAlpha, recoveredBeta = ...
...

print('We recovered the following LCG paramters:\n', 'Alpha:', recoveredAlpha, '\n Beta:', recoveredBeta)

assert recoveredAlpha is not None and recoveredBeta is not None, "Your solveModularSystem call is incorrect"

# Given alpha and beta, use recoverPreviousOutput(alpha,beta,N,currentOutput) to find b (s_2)!
recoveredB = ...

print('We recovered the previous LCG output, which is Bob\'s secret:\n', recoveredB)

# Given b and g^a (variable g_a), recover g^ab!
recoveredSecret = ...

print('We recovered the following shared secret key:\n', recoveredSecret)

# Now that we have the secret, we can decrypt the message they encrypted using that as their AES key!
# Use decrypt(key, interceptedMessage) to recover it.
interceptedMessage = b"o<\xb6\x10\xc9B$\xe18\xba\x13\xe2-'Vo\x18:%\x8c\x0f\x01\xc5cj\x07=%\xbe\x8a\x99\x1e"
decryptedMessage = ...

print('We decrypted the following message:\n', decryptedMessage)

In [ ]:
grader.check("q2_1")

# Hashing as a CSPRNG

While the linear congruential generator is evidently garbage for use in cryptography, we can actually build a pretty good CSPRNG just by using a secure hash function. 

If we accept that the hash function's output is computationally indistinguishable from random (as any good hash function should be), then we can use its output as the state, and hash that when asked for new numbers.

$$s_{i+1} = H(s_i)$$

We can cut off the most significant bits if needed. NOTE: This is nowhere close to an optimal hash-based CSPRNG, this is just for demonstration purposes!

In [ ]:
## This code is given to you!

def hashPRNG(seed, bits, cap):
    assert bits <= 256, "Can only output at most 256 bits"
    state = seed
                
    while cap > 0:
        state = int(H(state),16)  
        yield state & ((1 << bits) - 1)
        cap -= 1

In [ ]:
hashP = hashPRNG(np.random.randint(0,2**30), 5, 10000)
plt.bar(range(2**5), returnCounts(hashP, range(2**5)).values())
plt.show()

Looks reasonably random to me! This has complete rollback resistance as well, since rolling back requires inverting a hash function.

Congrats on finishing Lab 7!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

Once you have generated the zip file, go to the Gradescope page for this assignment to submit.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)